```{thebe-button}
```


# 02: Data Cleaning

**Objective:** Identify and handle missing or invalid values, detect outliers, and standardize data for our recidivism dataset.

**Key Steps:**

<table width="100%">
  <tr>
    <td style="vertical-align: top; text-align: left; width: 60%; padding-right: 20px;">
      <ol style="font-size: 20px; line-height: 1.4;">
        <li>Identify &amp; quantify missing data</li>
        <li>Handle missing values (imputation, removal, flagging)</li>
        <li>Validate &amp; correct data types</li>
        <li>Detect &amp; treat outliers</li>
        <li>Standardize &amp; normalize</li>
        <li>Document transformations</li>
      </ol>
    </td>
    <td style="vertical-align: top; text-align: left; width: 40%;">
      <!-- relative path to cleaning.png -->
      <img src="../slides/cleaning.png" alt="Data Cleaning Image" width="1000" />
    </td>
  </tr>
</table>

---
<audio controls src="../audio/Cleaning.m4a">

Now that we’ve laid out our data‑cleaning roadmap, let’s put it into practice with a real dataset. For the rest of this lesson, we’ll work with the **COMPAS recidivism data** (`compas‑scores‑raw.csv`), which contains demographic and risk‑assessment scores for individuals screened by the COMPAS tool. 

We’ll start by running through each cleaning step in **R**, then you are encouraged to repeat the same process in **Python** so you can see both workflows side by side.

## R

Since this is your first time, you’ll need to download all of the little helper programs (called “packages”) we’ll use—just run one simple command to get them all at once:



In [1]:
# only run this block your first time doing this training.


pkgs <- scan("../requirements_R.txt", what = "")
install.packages(
  pkgs,
  repos        = "https://cloud.r-project.org",
  dependencies = TRUE
)


Installing packages into 'C:/Users/demoore/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



Warning message:
"dependency 'RDCOMClient' is not available"


package 'tidyverse' successfully unpacked and MD5 sums checked


package 'lubridate' successfully unpacked and MD5 sums checked


package 'DescTools' successfully unpacked and MD5 sums checked


package 'corrplot' successfully unpacked and MD5 sums checked



The downloaded binary packages are in
	C:\Users\demoore\AppData\Local\Temp\RtmpKyML6I\downloaded_packages


#### Loading the tidyverse Package

Before we can start working with our data in R, we need to load a set of helpful tools called the **tidyverse**. The tidyverse gives us simple, consistent commands for:

- **Reading** data files (for example, Excel or CSV tables)  
- **Filtering** and **arranging** rows of data  
- **Summarizing** and **grouping** information  
- **Creating** basic charts and graphs  

Even if you’ve never done any data work before, this one line will make all of those functions available:



In [2]:
library(tidyverse)

Warning message:
"package 'tidyverse' was built under R version 4.4.3"


Warning message:
"package 'ggplot2' was built under R version 4.4.3"


Warning message:
"package 'tibble' was built under R version 4.4.3"


Warning message:
"package 'tidyr' was built under R version 4.4.3"


Warning message:
"package 'readr' was built under R version 4.4.3"


...

Warning message:
"package 'stringr' was built under R version 4.4.3"


Warning message:
"package 'forcats' was built under R version 4.4.3"


Warning message:
"package 'lubridate' was built under R version 4.4.3"


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


##### It’s OK to See Warning Messages

When you load a package in R, you might see warnings like:
```r
Warning message:
"package 'tidyverse' was built under R version 4.4.3
```

These messages are harmless—they simply mean the package was compiled under a slightly different R release than the one you’re using. They do **not** indicate an error in your code or your analysis.

You can safely ignore these warnings and continue working:
- Your functions will still run as expected  
- Your data cleaning and analysis steps are unaffected  




#### Now, once you run library(tidyverse), you can:

- Use `read_csv("myfile.csv")` to import your data  
- Use `filter()` to narrow down the rows you care about  
- Use `select()` to pick the columns you want to keep  
- And much more—all with clear, English‑like commands


### 2.1 Identify & Quantify Missing Data

First, load the data and get a sense of where values are missing.

In [3]:
# Load the COMPAS scores data into R
df <- read_csv("../data/compas_scores_raw.csv")

Rows: 24272 Columns: 24


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): Agency_Text, LastName, FirstName, MiddleName, Sex_Code_Text, Ethni...
dbl  (9): Person_ID, AssessmentID, Case_ID, ScaleSet_ID, RecSupervisionLevel...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Finally, we call head(df) on the DataFrame to display the first few rows. This gives us a quick peek at the structure and contents of our dataset, including column names and sample values.

In [4]:
head(df)

Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,ScaleSet_ID,⋯,MaritalStatus,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,Age
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
50844,57167,51950,PRETRIAL,Fisher,Kevin,NA,Male,Caucasian,22,⋯,Single,1,Low,18,Risk of Failure to Appear,15.00,1,Low,New,20
50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NA,Male,Caucasian,22,⋯,Married,1,Low,18,Risk of Failure to Appear,19.00,3,Low,New,28
50855,57181,51963,PRETRIAL,DAYES,DANIEL,NA,Male,African-American,22,⋯,Single,4,High,8,Risk of Recidivism,0.18,8,High,New,18
50855,57181,51963,PRETRIAL,DAYES,DANIEL,NA,Male,African-American,22,⋯,Single,4,High,18,Risk of Failure to Appear,13.00,1,Low,New,18
50850,57176,51958,PRETRIAL,Debe,Mikerlie,George,Female,African-American,22,⋯,Significant Other,2,Medium,18,Risk of Failure to Appear,11.00,1,Low,New,18
50839,57162,51945,PRETRIAL,McLaurin,Stephanie,Nicole,Female,African-American,22,⋯,Single,1,Low,18,Risk of Failure to Appear,16.00,2,Low,New,27


> **What to look for in the output:**
> - **Column names** such as `Person_ID`, `Case_ID`, `LastName`, `FirstName`, `MiddleName`, `Sex_Code_Text`, `Ethnic_Code_Text`, `RawScore`, `DecileScore`, `ScoreText`, `Age`  
> - **Data types** (numeric vs. categorical) and any surprising or missing values  
> - **Structure** of the dataset—how it's organized before we begin cleaning and analysis  

Now that we’ve successfully loaded the COMPAS dataset into our `df` object, it’s time to get a quick overview of its structure. In the next step, we’ll check how big the table is and see where any missing values might be hiding.

#### Checking Dataset Size and Missing Data

In this step, we first look at the overall size of our dataset and then count any empty or missing values:
 

In [5]:
str(df)

spc_tbl_ [24,272 × 24] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Person_ID              : num [1:24272] 50844 50848 50855 50855 50850 ...
 $ AssessmentID           : num [1:24272] 57167 57174 57181 57181 57176 ...
 $ Case_ID                : num [1:24272] 51950 51956 51963 51963 51958 ...
 $ Agency_Text            : chr [1:24272] "PRETRIAL" "PRETRIAL" "PRETRIAL" "PRETRIAL" ...
 $ LastName               : chr [1:24272] "Fisher" "KENDALL" "DAYES" "DAYES" ...
 $ FirstName              : chr [1:24272] "Kevin" "KEVIN" "DANIEL" "DANIEL" ...
 $ MiddleName             : chr [1:24272] NA NA NA NA ...
 $ Sex_Code_Text          : chr [1:24272] "Male" "Male" "Male" "Male" ...
 $ Ethnic_Code_Text       : chr [1:24272] "Caucasian" "Caucasian" "African-American" "African-American" ...
 $ ScaleSet_ID            : num [1:24272] 22 22 22 22 22 22 22 22 22 22 ...
 $ ScaleSet               : chr [1:24272] "Risk and Prescreen" "Risk and Prescreen" "Risk and Prescreen" "Risk and Prescreen" ...
 $ Language 

#### What `str(df)` Shows You


- **Tibble [24,272 × 24]**  
  Your data has **24,272 rows** (individual records) and **24 columns** (fields).

- **Column lines**  
  Each line looks like  
  ```
  $ ColumnName : type [1:24272] example values…
  ```  
  - **ColumnName**: the name of the field (e.g. `Person_ID`)  
  - **type**: `num` means numbers, `chr` means text  
  - **[1:24272]**: shows values exist for every row  
  - **example values…**: the first few entries you’ll see

- **`NA`** means a missing value (no data) in that spot.

- **Why this matters**  
  - You immediately know how big your dataset is  
  - You see which columns are numeric vs text  
  - You get a quick look at the first few values and spot missing entries before analysis  


### 2.2 Handle Missing Values

#### Counting Missing Values

Let’s see which columns have missing data and how many blanks each contains:



In [6]:
# Count missing values per column
missing <- colSums(is.na(df))

# Show only columns that have any missing values
cat("Missing values per column:\n")
print(missing[missing > 0])

Missing values per column:


MiddleName  ScoreText        Age 
     17942         45         56 


#### Options for Handling Missing Values
- **Removal:** drop rows or columns with too many nulls  
- **Imputation:** fill with mean/median or a constant  

##### MiddleName: 17,942 missing values  
  Most records don’t include a middle name—since this field isn’t critical to our analysis, we’ll drop the entire column.

##### ScoreText: 45 missing values  
  A small number of records lack the textual risk label. We’ll remove any rows where `ScoreText` is missing, since we need that label for downstream analyses.

##### Age: 56 missing values
  A handful of entries are missing age information. Since age is important for our analysis, we’ll impute these missing ages with the median age value.

By printing only `missing[missing > 0]`, we focus on the columns that actually have missing entries, allowing us to target our cleaning efforts precisely.  

Below we’ll:
1. Drop columns with >50% missing  
2. Impute `Age` with the median  



##### Step 1: Drop Mostly Empty Columns

Our dataset has a “MiddleName" column that’s almost entirely blank. To avoid clutter, we remove any column where more than half of its values are missing. This will automatically drop “MiddleName” and any other mostly empty fields.


In [7]:
# Calculate the minimum non-missing entries (50% of rows)
threshold <- nrow(df) * 0.5

# Keep only columns with at least 'threshold' non-NA values
df <- df[, colSums(!is.na(df)) >= threshold]

##### Step 2: Remove Rows with Blank `ScoreText` and Fill Missing `Age`

First, we drop any rows where `ScoreText` is blank, since those entries don’t tell us whether someone was classified as “Low,” “Medium,” or “High” risk. After that, we flag rows with missing `Age`, compute the median age, and fill those gaps.


In [8]:
# 1. Remove rows where ScoreText is blank
df <- df[trimws(df$ScoreText) != "", ]

# 2. Flag rows where Age was missing
df$age_missing <- is.na(df$Age)

# 3. Compute median Age and fill missing values
median_age <- median(df$Age, na.rm = TRUE)
df$Age[is.na(df$Age)] <- median_age

##### Now we have:
- Removed all rows lacking a valid risk category in `ScoreText`.  
- Marked originally missing `Age` values in a new `age_missing` column.  
- Filled those missing ages with the dataset’s median age.

### 2.3 Validate & Correct Data Types

Before we go further, let’s make sure each column uses the right data type:

- Numbers as integers or floats  
- Dates as datetime objects  
- Text fields we’ll analyze categorically as “category”

In [9]:
# 1. Convert RawScore, DecileScore and Age to integers
df$RawScore    <- as.integer(df$RawScore)
df$DecileScore <- as.integer(df$DecileScore)
df$Age         <- as.integer(df$Age)

# 2. Convert text fields to factors
text_cols <- c(
  "ScoreText", "Sex_Code_Text", "Ethnic_Code_Text",
  "Language", "MaritalStatus", "RecSupervisionLevelText"
)
df[text_cols] <- lapply(df[text_cols], factor)

# 4. Verify types
str(df)


tibble [24,272 × 24] (S3: tbl_df/tbl/data.frame)
 $ Person_ID              : num [1:24272] 50844 50848 50855 50855 50850 ...
 $ AssessmentID           : num [1:24272] 57167 57174 57181 57181 57176 ...
 $ Case_ID                : num [1:24272] 51950 51956 51963 51963 51958 ...
 $ Agency_Text            : chr [1:24272] "PRETRIAL" "PRETRIAL" "PRETRIAL" "PRETRIAL" ...
 $ LastName               : chr [1:24272] "Fisher" "KENDALL" "DAYES" "DAYES" ...
 $ FirstName              : chr [1:24272] "Kevin" "KEVIN" "DANIEL" "DANIEL" ...
 $ Sex_Code_Text          : Factor w/ 2 levels "Female","Male": 2 2 2 2 1 1 2 2 1 2 ...
 $ Ethnic_Code_Text       : Factor w/ 9 levels "African-Am","African-American",..: 5 5 2 2 2 2 6 6 5 2 ...
 $ ScaleSet_ID            : num [1:24272] 22 22 22 22 22 22 22 22 22 22 ...
 $ ScaleSet               : chr [1:24272] "Risk and Prescreen" "Risk and Prescreen" "Risk and Prescreen" "Risk and Prescreen" ...
 $ Language               : Factor w/ 2 levels "English","Spanish": 1 1

> **What we’ve done:**  
> - Forced `RawScore`, `DecileScore`, and `Age` into integer form   
> - Marked risk categories and demographic fields as categorical  
> - Verified the changes by once again inspecting the structure of the 'df' dataframe 

### Step 2.4: Find and Remove Extreme Values (Outliers) in `RawScore`

Even smart data can hide a few extreme values—called **outliers**—that skew our insights. We’ll clean those out by:

1. **Sorting** all `RawScore` values and finding the 25th percentile (Q1) and 75th percentile (Q3).  
2. Calculating the **interquartile range (IQR)** = Q3 − Q1, which covers the middle 50% of the data.  
3. Defining an **acceptable range** as   [Q1 − 3×IQR, Q3 + 3×IQR]
4. **Keeping** only rows with `RawScore` inside that range and dropping the rest.

This preserves most of the data while removing the very highest or lowest scores that could mislead our analysis.


In [10]:
# 1. Calculate Q1 (25th percentile) and Q3 (75th percentile) for RawScore
Q1 <- quantile(df$RawScore, 0.25, na.rm = TRUE)
Q3 <- quantile(df$RawScore, 0.75, na.rm = TRUE)

# 2. Compute the IQR
IQR_value <- Q3 - Q1

# 3. Define acceptable lower and upper bounds
lower_bound <- Q1 - 3 * IQR_value
upper_bound <- Q3 + 3 * IQR_value

# 4. Filter the data frame to keep only non-outliers
df_clean <- df[df$RawScore >= lower_bound & df$RawScore <= upper_bound, ]

# 5. Report how many rows we kept versus removed
cat("Rows before cleaning: ", nrow(df), "\n")
cat("Rows after removing outliers: ", nrow(df_clean), "\n")


Rows before cleaning:  24272 


Rows after removing outliers:  24263 


> **In plain terms:**  
> - We measured how wide the middle 50% of scores is.  
> - We dropped any scores more than three times that range away from the center.  
> - Now `df_clean` has most of our original rows minus the extreme cases that could distort averages or trends.

### Step 2.5: Standardize & Normalize Key Variables

Some analysis methods work best when numbers share a common scale. In R we can use:

- **`scale()`** for z-score standardization  
- A simple formula for min-max scaling  

Here’s what we’ll do on our cleaned data frame `df_clean`:

1. **Z-score Standardization** of `Age`  
   - Subtract the mean age, then divide by the standard deviation  
   - Creates a new column `age_z` where most values fall between –3 and +3  

2. **Min-Max Scaling** of `RawScore`  
   - Rescales values to lie between 0 (lowest score) and 1 (highest score)  
   - Creates a new column `rawscore_scaled` that preserves relative differences  


In [11]:

# 1. Z-score standardization on Age
df_clean$age_z <- as.numeric(scale(df_clean$Age))

# 2. Min-Max scaling on RawScore
raw_min <- min(df_clean$RawScore, na.rm = TRUE)
raw_max <- max(df_clean$RawScore, na.rm = TRUE)
df_clean$rawscore_scaled <- (df_clean$RawScore - raw_min) / (raw_max - raw_min)

# 3. Peek at the new columns
head(df_clean[, c("Age", "age_z", "RawScore", "rawscore_scaled")])


Age,age_z,RawScore,rawscore_scaled
<int>,<dbl>,<int>,<dbl>
20,-1.0581198,15,0.3333333
28,-0.3877174,19,0.4222222
18,-1.2257204,0,0.0000000
18,-1.2257204,13,0.2888889
18,-1.2257204,11,0.2444444
27,-0.4715177,16,0.3555556


## Interpreting Our Transformed Data



| Age | age_z | RawScore | rawscore_scaled |
| --- | ----- | -------- | --------------- |
| 20  | -1.057784 | 15       | 0.3333      |
| 28  | -0.387237 | 19       | 0.4222      |
| 18  | -1.225421 | 0        | 0.0000      |
| 18  | -1.225421 | 13       | 0.2888      |

### What each column is

- **Age**  
  The actual age in years

- **age_z**  
  A “z‑score” shows how far each age is from the average age, measured in standard units  
  - 0 means exactly average  
  - Negative means below average  
  - Positive means above average  
  - −-1.05778 means about -1.05778 units younger than average

- **RawScore**  
  The original score before any changes (for example a test result or rating)

- **rawscore_scaled**  
  The RawScore rescaled to lie between 0 and 1  
  - 0 is the lowest RawScore in our group  
  - 1 is the highest RawScore in our group  
  - 0.03 means very close to the lowest  
  - 0.29 means 29 percent of the way from lowest to highest

### Why we do this

1. **Fair comparison**  
   When columns use very different units or ranges our analysis can get biased. Standardizing and scaling puts all numbers on the same footing.

2. **Better results**  
   Many data tools and machine learning methods work best when inputs live on the same scale.

3. **Clear interpretation**  
   - Z‑scores tell us how far a value is from the average in comparable units  
   - Scaled scores between 0 and 1 make it easy to see relative position without worrying about original units

---

In plain terms, we’ve “translated” every number so they all speak the same language. That helps our next analysis steps work properly and gives you a fair way to compare apples to apples.

### Step 2.6: Record Your Cleaning Steps

Finally, it’s best practice to keep a log of every transformation. Below we build a simple dictionary summarizing what we did:

- Which columns we dropped because they were mostly empty  
- The median age we used for imputation  
- The bounds we used to remove outliers in `RawScore`  
- Which columns we standardized and normalized  

At the end, we print this log in a clear, readable format.

In [12]:
# 1. Define each step and its details
steps <- c(
  "Dropped Columns",
  "Imputed Age",
  "Outlier Bounds (RawScore)",
  "Standardized",
  "Normalized"
)

details <- list(
  names(missing[missing > threshold]),      # columns dropped
  median_age,                               # age used for imputation
  c(lower_bound, upper_bound),              # outlier bounds
  "age_z",                                  # standardized column
  "rawscore_scaled"                         # normalized column
)

# 2. Assemble into a data frame with a list-column
log_df <- data.frame(
  Step    = steps,
  Details = I(details),
  stringsAsFactors = FALSE
)

# 3. Display the log
print(log_df)


                       Step      Details
1           Dropped Columns   MiddleName
2               Imputed Age           29
3 Outlier Bounds (RawScore)      -11, 45
4              Standardized        age_z
5                Normalized rawscore....


 **How to read this table:**  
 - **Step:** what we did  
 - **Details:** the exact values or columns affected by that step  


> **Why this matters:**  
> A clear transformation log makes your work reproducible and lets others (or future you) understand exactly how the data was prepared before any analysis or modeling.

## Next Steps: Continue in R or Switch to Python

Our data is now clean and properly formatted. You have two options:

1. **Try the Python version** of this cleaning workflow by opening `02_data_cleaning_Python.ipynb`.  
2. **Move on to summary statistics in R** by opening `03_summary_statistics_R.ipynb`.  

Choose the notebook that matches your preferred language, and let’s continue exploring our recidivism dataset!